## Demo LBM CaImAn Pipeline

Processing and exploring the LBM dataset located [here](https://rockefeller-my.sharepoint.com/my?id=%2Fpersonal%2Ffoconnell%5Frockefeller%5Fedu%2FDocuments%2FMBO%2Fdatasets%2Fdemas%5F2023&FolderCTID=0x012000228B6C10491B3546BA7A3CF92268F533I)

In [3]:
import os
from pathlib import Path
import logging

# itertools.product makes it easy to loop through parameter variants
# basically, it's easier to read that deeply nested for loops
from copy import deepcopy
from itertools import product

import mesmerize_core as mc
from mesmerize_viz import *
from mesmerize_core.caiman_extensions.cnmf import cnmf_cache

import caiman as cm
from caiman.source_extraction.cnmf import cnmf, params
from caiman.motion_correction import high_pass_filter_space
from caiman.summary_images import correlation_pnr

import napari

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import zarr
from ipywidgets import IntSlider, VBox
from sidecar import Sidecar

try:
    import cv2
    cv2.setNumThreads(0)
except():
    pass

if os.name == "nt":
    # disable the cache on windows, this will be automatic in a future version
    cnmf_cache.set_maxsize(0)

pd.options.display.max_colwidth = 120

logger = logging.getLogger("caiman")
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
log_format = logging.Formatter("%(relativeCreated)12d [%(filename)s:%(funcName)10s():%(lineno)s] [%(process)d] %(message)s")
handler.setFormatter(log_format)
logger.addHandler(handler)

logging.getLogger("caiman").setLevel(logging.WARNING)

In [4]:
parent_path = Path().home() / "caiman_data" / "animal_01" / "session_01"

batch_path = parent_path / 'batch.pickle'
mc.set_parent_raw_data_path(str(parent_path))

# you could alos load the registration batch and
# save this patch in a new dataframe (saved to disk automatically)

try:
    df = mc.load_batch(batch_path)
except (IsADirectoryError, FileNotFoundError):
    df = mc.create_batch(batch_path)

df=df.caiman.reload_from_disk()
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,extracted_plane_1,tiff\extracted_plane_1.tif,"{'main': {'var_name_hdf5': 'mov', 'max_shifts': (10, 10), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_...",{'mean-projection-path': b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-a9a5-4965-be7c-e6779e854328_mean_projection.n...,2024-09-26T11:56:54,2024-09-26T12:02:55,77.3 sec,None,b32f41bf-a9a5-4965-be7c-e6779e854328
1,cnmf,cnmf_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-a9a5-4965-be7c-e6779e854328-extracted_plane_1_els__d1_583_d2_536_d3_1_...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'decay_time': 0.4, 'strides': (48, 48), 'overlaps': (24, 24), 'max_shifts':...",{'mean-projection-path': a057e39e-a2df-41d3-8217-83c9cd7ffb6d\a057e39e-a2df-41d3-8217-83c9cd7ffb6d_mean_projection.n...,2024-09-26T16:26:20,2024-09-26T16:28:48,143.18 sec,None,a057e39e-a2df-41d3-8217-83c9cd7ffb6d
2,cnmf,extracted_plane_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-a9a5-4965-be7c-e6779e854328-extracted_plane_1_els__d1_583_d2_536_d3_1_...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'decay_time': 0.4, 'strides': (48, 48), 'overlaps': (24, 24), 'max_shifts':...",{'mean-projection-path': 5cb543ec-5358-4b35-83cf-bfd19fa06a68\5cb543ec-5358-4b35-83cf-bfd19fa06a68_mean_projection.n...,2024-09-27T14:48:59,2024-09-27T14:50:30,80.14 sec,None,5cb543ec-5358-4b35-83cf-bfd19fa06a68
3,cnmf,extracted_plane_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-a9a5-4965-be7c-e6779e854328-extracted_plane_1_els__d1_583_d2_536_d3_1_...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'decay_time': 0.4, 'strides': (48, 48), 'overlaps': (24, 24), 'max_shifts':...",{'mean-projection-path': df52a609-d986-4bc0-a0a2-526cbfda057d\df52a609-d986-4bc0-a0a2-526cbfda057d_mean_projection.n...,2024-09-27T14:48:59,2024-09-27T14:51:46,75.93 sec,None,df52a609-d986-4bc0-a0a2-526cbfda057d
4,cnmf,extracted_plane_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-a9a5-4965-be7c-e6779e854328-extracted_plane_1_els__d1_583_d2_536_d3_1_...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'decay_time': 0.4, 'strides': (48, 48), 'overlaps': (24, 24), 'max_shifts':...",{'mean-projection-path': b68a8efe-1a3b-4392-b6e7-416312f1f945\b68a8efe-1a3b-4392-b6e7-416312f1f945_mean_projection.n...,2024-09-27T14:48:59,2024-09-30T10:30:15,96.62 sec,None,b68a8efe-1a3b-4392-b6e7-416312f1f945
5,cnmf,extracted_plane_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-a9a5-4965-be7c-e6779e854328-extracted_plane_1_els__d1_583_d2_536_d3_1_...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'decay_time': 0.4, 'strides': (48, 48), 'overlaps': (24, 24), 'max_shifts':...",{'mean-projection-path': 15e6fb81-fac4-457b-96da-dbb8e2d7b669\15e6fb81-fac4-457b-96da-dbb8e2d7b669_mean_projection.n...,2024-09-27T14:48:59,2024-09-30T11:32:09,89.61 sec,None,15e6fb81-fac4-457b-96da-dbb8e2d7b669
6,cnmf,k=40,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-a9a5-4965-be7c-e6779e854328-extracted_plane_1_els__d1_583_d2_536_d3_1_...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'decay_time': 0.4, 'strides': (48, 48), 'overlaps': (24, 24), 'max_shifts':...",{'mean-projection-path': abba269d-f2f3-467b-ba40-f1e33aea8461\abba269d-f2f3-467b-ba40-f1e33aea8461_mean_projection.n...,2024-09-30T14:02:10,2024-09-30T14:05:00,165.71 sec,None,abba269d-f2f3-467b-ba40-f1e33aea8461
7,cnmf,cnmf_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-a9a5-4965-be7c-e6779e854328-extracted_plane_1_els__d1_583_d2_536_d3_1_...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'decay_time': 0.4, 'strides': (48, 48), 'overlaps': (24, 24), 'max_shifts':...",{'mean-projection-path': 9777376d-092c-43db-b14a-c9be1e142e5c\9777376d-092c-43db-b14a-c9be1e142e5c_mean_projection.n...,2024-09-30T14:36:58,2024-09-30T14:41:40,161.74 sec,None,9777376d-092c-43db-b14a-c9be1e142e5c
8,cnmf,cnmf_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-a9a5-4965-be7c-e6779e854328-extracted_plane_1_els__d1_583_d2_536_d3_1_...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'decay_time': 0.4, 's

## Decay Times

  	Max dF/F 	Half-rise time (ms) 	Time to peak (ms) 	Half-decay time (ms)
    jGCaMP7f (control) 	0.21±0.1 	24.8±6.6 	99.5±30.2 	181.9±76.0
    jGCaMP8f 	0.41±0.12 	7.1±0.74 	24.8±6.1 	67.4±11.2
    jGCaMP8m 	0.76±0.22 	7.1±0.61 	29.0±11.2 	118.3±13.2
    jGCaMP8s 	1.11±0.22 	10.1±0.86 	57.0±12.9 	306.7±32.2
    jGCaMP8.712* 	0.66±0.18 	10.9±1.24 	41.6±8.1 	94.8±13.3

In [6]:
k = 150

# general dataset-dependent parameters
fr = 9.62                   # imaging rate in frames per second
decay_time = 0.4            # length of a typical transient in seconds
dxy = (1., 1.)              # spatial resolution in x and y in (um per pixel)

# CNMF parameters for source extraction and deconvolution
p = 2                       # order of the autoregressive system (set p=2 if there is visible rise time in data)
gnb = 1                     # number of global background components (set to 1 or 2)
merge_thr = 0.80            # merging threshold, max correlation allowed
bas_nonneg = True           # enforce nonnegativity constraint on calcium traces (technically on baseline)
rf = 300                     # half-size of the patches in pixels (patch width is rf*2 + 1)
stride_cnmf = 15            # amount of overlap between the patches in pixels (overlap is stride_cnmf+1)
gSig = np.array([6, 6])     # expected half-width of neurons in pixels (Gaussian kernel standard deviation)
gSiz = None #2*gSig + 1     # Gaussian kernel width and hight
method_init = 'greedy_roi'  # initialization method (if analyzing dendritic data see demo_dendritic.ipynb)
ssub = 1                    # spatial subsampling during initialization
tsub = 1                    # temporal subsampling during intialization

# parameters for component evaluation
min_SNR = 1.4               # signal to noise ratio for accepting a component
rval_thr = 0.80             # space correlation threshold for accepting a component

params_cnmf = {
    'main': {
        'fr': fr,
        'dxy': dxy,
        'decay_time': decay_time,
        'p': p,
        'nb': gnb,
        'rf': rf,
        'K':  k,
        'gSig': gSig,
        'gSiz': gSiz,
        'stride': stride_cnmf,
        'method_init': method_init,
        'rolling_sum': True,
        'use_cnn': False,
        'ssub': ssub,
        'tsub': tsub,
        'merge_thr': merge_thr,
        'bas_nonneg': bas_nonneg,
        'min_SNR': min_SNR,
        'rval_thr': rval_thr,
    },
    'refit': True
}

In [2]:
result = 9.2e4 * 20e-9 * (1 * 40)**2
result

2.944

In [7]:
# variants of several parameters
K_variants = [40, 60, 80]
merge_thr_variants = [0.7, 0.8, 0.95]
r_val_variants = [0.7, 0.8]
g_sig_variants = [6, 8, 10]

# always use deepcopy like before
new_params_cnmf = deepcopy(params_cnmf)

# create a parameter grid
parameter_grid = product(g_sig_variants, K_variants, r_val_variants, merge_thr_variants)

# a single for loop to go through all the various parameter combinations
for gSig, k, r_val, merge_val in parameter_grid:
    # deep copy params dict just like before
    new_params_cnmf = deepcopy(new_params_cnmf)

    new_params_cnmf["main"]["gSig"] = [gSig, gSig]
    new_params_cnmf["main"]["K"] = k
    new_params_cnmf["main"]["merge_thr"] = merge_val
    new_params_cnmf["main"]["rval_thr"] = r_val

    # add param combination variant to batch
    df.caiman.add_item(
        algo="cnmf",
        item_name=df.iloc[1]["item_name"],  # good mcorr item
        input_movie_path=df.iloc[0],
        params=new_params_cnmf
    )

In [9]:
for i, row in df.iterrows():
    if row["outputs"] is not None: # item has already been run
        continue # skip

    process = row.caiman.run()

    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = df.caiman.reload_from_disk()

Running 9777376d-092c-43db-b14a-c9be1e142e5c with local backend
************************************************************************

Starting CNMF item:
algo                                                                                                                                   cnmf
item_name                                                                                                                            cnmf_1
input_movie_path    b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-a9a5-4965-be7c-e6779e854328-extracted_plane_1_els__d1_583_d2_536_d3_1_...
params              {'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'decay_time': 0.4, 'strides': (48, 48), 'overlaps': (24, 24), 'max_shifts':...
outputs                                                                                                                                None
added_time                                                                                                              2024-09-30T14:36:58
ra

      472727 [cluster.py:setup_cluster():225] [24368] The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
      472727 [cluster.py:setup_cluster():225] [24368] The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
      472830 [params.py:change_params():1151] [24368] In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
      472830 [params.py:change_params():1151] [24368] In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)


making memmap


      482847 [cluster.py:setup_cluster():225] [24368] The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
      482847 [cluster.py:setup_cluster():225] [24368] The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman


performing CNMF
fitting images
refitting
performing eval
Running 88177367-8fbf-499b-940e-1fb9c331b499 with local backend
************************************************************************

Starting CNMF item:
algo                                                                                                                                   cnmf
item_name                                                                                                                            cnmf_1
input_movie_path    b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-a9a5-4965-be7c-e6779e854328-extracted_plane_1_els__d1_583_d2_536_d3_1_...
params              {'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'decay_time': 0.4, 'strides': (48, 48), 'overlaps': (24, 24), 'max_shifts':...
outputs                                                                                                                                None
added_time                                                                           

...

performing CNMF
fitting images
refitting
performing eval
Running 4b6a7355-508e-4505-84ed-95ef3db5e81b with local backend
************************************************************************

Starting CNMF item:
algo                                                                                                                                   cnmf
item_name                                                                                                                            cnmf_1
input_movie_path    b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-a9a5-4965-be7c-e6779e854328-extracted_plane_1_els__d1_583_d2_536_d3_1_...
params              {'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'decay_time': 0.4, 'strides': (48, 48), 'overlaps': (24, 24), 'max_shifts':...
outputs                                                                                                                                None
added_time                                                                           

     4312881 [cluster.py:setup_cluster():225] [24368] The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
     4312881 [cluster.py:setup_cluster():225] [24368] The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
     4312983 [params.py:change_params():1151] [24368] In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
     4312983 [params.py:change_params():1151] [24368] In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)


making memmap


     4323211 [cluster.py:setup_cluster():225] [24368] The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
     4323211 [cluster.py:setup_cluster():225] [24368] The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman


performing CNMF
fitting images


In [15]:
num_neurons = (df.iloc[i].cnmf.get_temporal() for i, row in df.iterrows() if row['algo'] == 'cnmf')
[len(x) for x in num_neurons]

[1314,
 188,
 208,
 357,
 413,
 1783,
 1301,
 1783,
 2372,
 1301,
 1783,
 2372,
 1820,
 2681,
 3639,
 1820,
 2681,
 3639,
 2518,
 3664,
 4941,
 2518,
 3664]

In [49]:
out = df.iloc[1].cnmf.get_output()
Yr = np.mean(df.iloc[0].mcorr.get_output(), axis=0)

In [56]:
# calculate stride and overlap from parameters
# cnmf_patch_width = out.params.patch['rf']
# cnmf_patch_overlap = out.params.patch['stride']

out.params.patch

{'border_pix': 0,
 'del_duplicates': False,
 'in_memory': True,
 'low_rank_background': True,
 'memory_fact': 1,
 'n_processes': 31,
 'nb_patch': 1,
 'only_init': False,
 'p_patch': 0,
 'remove_very_bad_comps': False,
 'rf': None,
 'skip_refinement': False,
 'p_ssub': 2,
 'stride': 10,
 'p_tsub': 2}

In [ ]:
from caiman import load_memmap

In [1]:
Yr, dims, T = load_memmap(str(df.iloc[1].cnmf.get_m), mode=mode)

NameError: name 'load_memmap' is not defined

In [ ]:

cnmf_patch_stride = cnmf_patch_width - cnmf_patch_overlap
print(f'Patch width: {cnmf_patch_width} , Stride: {cnmf_patch_stride}, Overlap: {cnmf_patch_overlap}');


In [50]:
Yr.shape

(583, 536)

In [34]:
out.__dict__

{'dview': None,
 'dims': (583, 536),
 'empty_merged': None,
 'skip_refinement': False,
 'remove_very_bad_comps': False,
 'params': CNMFParams:
 
 data:
 
 {'caiman_version': '1.11.3',
  'decay_time': 0.4,
  'dims': None,
  'dxy': (1.0, 1.0),
  'fnames': None,
  'fr': 9.62,
  'last_commit': 'GITW-7a5f5454580c4ed3e2726178ce4c5fa9306076b5',
  'var_name_hdf5': 'mov'}
 
 init:
 
 {'K': 30,
  'SC_kernel': 'heat',
  'SC_nnn': 20,
  'SC_normalize': True,
  'SC_sigma': 1,
  'SC_thr': 0,
  'SC_use_NN': False,
  'alpha_snmf': 0.5,
  'center_psf': False,
  'gSig': array([6, 6]),
  'gSiz': array([13, 13]),
  'init_iter': 2,
  'kernel': None,
  'lambda_gnmf': 1,
  'maxIter': 5,
  'max_iter_snmf': 500,
  'method_init': 'greedy_roi',
  'min_corr': 0.85,
  'min_pnr': 20,
  'nIter': 5,
  'nb': 1,
  'normalize_init': True,
  'options_local_NMF': None,
  'perc_baseline_snmf': 20,
  'ring_size_factor': 1.5,
  'rolling_length': 100,
  'rolling_sum': True,
  'seed_method': 'auto',
  'sigma_smooth_snmf': (0.5

In [22]:
est.__dict__

{'A': <Compressed Sparse Column sparse matrix of dtype 'float64'
 	with 579349 stored elements and shape (312488, 1314)>,
 'C': array([[4472.99026926, 4365.36562107, 5595.72108568, ...,  141.40027243,
          118.57100293,   99.42755056],
        [   0.        ,    0.        , 4173.34844444, ..., 5099.3493474 ,
         4463.27577524, 4795.07249115],
        [3965.71293389, 3597.13128374, 4034.38615671, ..., 1936.96977384,
         1656.27008831, 2960.56356587],
        ...,
        [   0.        ,    0.        ,    0.        , ..., 1282.99838309,
          993.56003989,  769.09202546],
        [   0.        ,    0.        ,    0.        , ...,  110.29273966,
           89.63924066,   72.85333097],
        [   0.        ,    0.        ,    0.        , ..., 7417.69580277,
         6148.52758428, 5755.05429957]]),
 'f': array([[2000385.40348365, 1775443.23324557, 1713538.14611211, ...,
         1502142.83634028, 1507307.05741546, 1541275.92381877]]),
 'b': array([[0.00063404],
        

In [53]:
est.Y

AttributeError: 'Estimates' object has no attribute 'Y'

In [51]:
cm.utils.visualization.hv_view_patches(Yr, est.A, est.C, est.b, est.f, out.dims[0], out.dims[1], YrA)

ValueError: dimension mismatch